<a href="https://colab.research.google.com/github/sioulruble/GenDL/blob/main/advanced_architectures/LAB1_GENDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import gzip
import torch
import numpy as np
import torch.nn.functional as F
from torchsummary import summary

np.random.seed(42)

# Téléchargement du fichier de données
#!wget http://dihana.cps.unizar.es/~cadrete/gen/p1_data1.pkl.gz

# Chargement des données
with gzip.open('/home/p1_data1.pkl.gz', 'rb') as f:
    x_train, y_train, x_test, y_test = pickle.load(f)

x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255

# Création d'un dataset personnalisé
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

trainset = CustomDataset(x_train, y_train)
testset = CustomDataset(x_test, y_test)

# Définition du modèle
class Net(torch.nn.Module):
    def __init__(self, di=1, h1=16, h2=32, h3=64, C=2):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(di, h1, kernel_size=5, padding=2, stride=2)
        self.conv2 = torch.nn.Conv2d(h1, h2, kernel_size=3, padding=1, stride=2)
        self.fc1 = torch.nn.Linear(h2, h3)
        self.fc2 = torch.nn.Linear(h3, C)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.mean([2, 3])
        x = F.relu(self.fc1(x))
        o = self.fc2(x)
        return o

    def predict(self, x):
        return F.softmax(self.forward(x), dim=1)

# Initialisation du modèle et affichage du résumé
model = Net().cuda()
summary(model, (1, 32, 32))

# Paramètres d'entraînement
batch_size = 32
nb_its = 12
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
J = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.6, weight_decay=1e-5)
J_it = np.zeros(nb_its)

# Boucle d'entraînement
for i in range(nb_its):
    k = 0
    for x, y in trainloader:
        x = x.cuda()
        y = y.cuda()
        model.zero_grad()
        o = model(x)
        loss = J(o, y)
        loss.backward()
        optimizer.step()
        J_it[i] += loss.item()
        k += 1
    J_it[i] /= k
    print("it %d: J() = %.3f" % (i, J_it[i]))

# Test du modèle
x = torch.tensor(x_test, dtype=torch.float32).cuda()
o = model(x).detach().cpu().numpy()
o_max = np.argmax(o, axis=1)
err = np.sum(y_test != o_max)
print('Error %%: %f' % (err / len(y_test) * 100))






----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             416
            Conv2d-2             [-1, 32, 8, 8]           4,640
            Linear-3                   [-1, 64]           2,112
            Linear-4                    [-1, 2]             130
Total params: 7,298
Trainable params: 7,298
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.03
Estimated Total Size (MB): 0.08
----------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


it 0: J() = 0.695
it 1: J() = 0.692
it 2: J() = 0.679
it 3: J() = 0.671
it 4: J() = 0.568
it 5: J() = 0.341
it 6: J() = 0.258
it 7: J() = 0.191
it 8: J() = 0.147
it 9: J() = 0.111
it 10: J() = 0.088
it 11: J() = 0.077
Error %: 1.915114





# **Customized RNN from a CNN**


https://medium.com/@chen-yu/building-a-customized-residual-cnn-with-pytorch-471810e894ed


In [58]:
from torchviz import make_dot
from IPython.display import Image


import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, di=1, h1=16, h2=32, h3=64, C=2) :
        super().__init__()

        # Conv 5x5
        self.conv5x5_1 = nn.Conv2d(di, h1, kernel_size=5, padding='same', bias=False)
        self.bn1 = nn.BatchNorm2d(h1)

        self.conv5x5_2 = nn.Conv2d(h1, h2, kernel_size=5, padding='same', bias=False)
        self.bn2 = nn.BatchNorm2d(h2)

        # Conv 1x1
        self.conv1x1 = nn.Conv2d(h2, h3, kernel_size=1, bias=False)
        self.bn1x1 = nn.BatchNorm2d(h3)
        # Conv 3x3 sur la branche parallèle
        self.conv3x3_1 = nn.Conv2d(h2, h3, kernel_size=3, padding='same', bias=False)  # h2 en entrée, h3 en sortie
        self.bn3x3_1 = nn.BatchNorm2d(h3)
        self.conv3x3_2 = nn.Conv2d(h3, h3, kernel_size=3, padding='same', bias=False)
        self.bn3x3_2 = nn.BatchNorm2d(h3)

        self.relu = nn.ReLU()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(h3, C)

    def forward(self, x):
        x = self.bn1(self.conv5x5_1(x))
        x = self.relu(self.bn2(self.conv5x5_2(x)))

        branch1 = self.bn1x1(self.conv1x1(x))

        branch2 = self.bn3x3_1(self.conv3x3_1(x))
        branch2 = self.relu(self.bn3x3_2(self.conv3x3_2(branch2)))

        # Combiner les branches
        x = self.relu(branch1 + branch2)
        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        return x


    def predict(self, x):
        return self.relu(self.forward(x), dim=1)


# Initialisation et visualisation
residual_block = ResidualBlock().cuda()

x = torch.randn(1,1,32, 32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
residual_block = residual_block.to(device)
x = x.to(device)
summary(residual_block,(1,32, 32))

# Générer le graphe
y = residual_block(x)
# dot = make_dot(y, params=dict(residual_block.named_parameters()))

# # Sauvegarder le graphe en format PNG
# dot.render("residual_block", format="png")

# # Afficher l'image générée dans Colab
# Image("residual_block.png")

!pip install onnx
torch.onnx.export(residual_block,x,'model.onnx')



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             400
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 32, 32, 32]          12,800
       BatchNorm2d-4           [-1, 32, 32, 32]              64
              ReLU-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]           2,048
       BatchNorm2d-7           [-1, 64, 32, 32]             128
            Conv2d-8           [-1, 64, 32, 32]          18,432
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
             ReLU-13           [-1, 64, 32, 32]               0
AdaptiveAvgPool2d-14             [-1, 6

In [61]:
# Initialisation du modèle
model = ResidualBlock().cuda()

# Paramètres d'entraînement
batch_size = 32
nb_its = 12
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Boucle d'entraînement
for epoch in range(nb_its):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.3f}")
    # Test du modèle
x = torch.tensor(x_test, dtype=torch.float32).cuda()
o = model(x).detach().cpu().numpy()
o_max = np.argmax(o, axis=1)
err = np.sum(y_test != o_max)
print('Error %%: %f' % (err / len(y_test) * 100))

Epoch 1, Loss: 2.971
Epoch 2, Loss: 2.049
Epoch 3, Loss: 1.256
Epoch 4, Loss: 0.801
Epoch 5, Loss: 0.539
Epoch 6, Loss: 0.387
Epoch 7, Loss: 0.289
Epoch 8, Loss: 0.226
Epoch 9, Loss: 0.186
Epoch 10, Loss: 0.153
Epoch 11, Loss: 0.133
Epoch 12, Loss: 0.111
Error %: 0.362319


# **Attention Model**


